In [2]:
# Let's write a Python code that calculates the expected monetary value (EMV) and the expected utility for the investment decision problem.

import math

# Define the monetary outcomes
outcomes = {
    'stock_up_stock_up': 22,
    'stock_up_stock_down': 12.5,
    'stock_up_bank': 14.5,
    'stock_down_stock_up': 11,
    'stock_down_stock_down': 6,
    'stock_down_bank': 8,
    'bank_stock_up': 13.5,
    'bank_stock_down': 8,
    'bank_bank': 10.5
}

# Define the probabilities
prob_stock_up_first = 0.4
prob_stock_up_given_up = 0.35
prob_stock_up_given_down = 0.45

# Utility function
def utility(x):
    return math.sqrt(x / 1000)

# Calculate expected monetary value and utility for the second investment period
def calculate_second_period(stock_up):
    # If the first action is investing in stock and it went up
    if stock_up:
        emv_stock = prob_stock_up_given_up * outcomes['stock_up_stock_up'] + \
                    (1 - prob_stock_up_given_up) * outcomes['stock_up_stock_down']
        emv_bank = outcomes['stock_up_bank']
    # If the first action is investing in stock and it went down
    else:
        emv_stock = prob_stock_up_given_down * outcomes['stock_down_stock_up'] + \
                    (1 - prob_stock_up_given_down) * outcomes['stock_down_stock_down']
        emv_bank = outcomes['stock_down_bank']

    # Calculate utility
    utility_stock = utility(emv_stock)
    utility_bank = utility(emv_bank)

    return emv_stock, emv_bank, utility_stock, utility_bank

# Calculate expected monetary value and utility for the first investment period
def calculate_first_period():
    # Calculate for stock
    emv_stock_up, _, utility_stock_up, _ = calculate_second_period(stock_up=True)
    emv_stock_down, _, utility_stock_down, _ = calculate_second_period(stock_up=False)
    
    # EMV for choosing stock in the first period
    emv_stock_first = prob_stock_up_first * emv_stock_up + (1 - prob_stock_up_first) * emv_stock_down
    # Utility for choosing stock in the first period
    utility_stock_first = prob_stock_up_first * utility_stock_up + (1 - prob_stock_up_first) * utility_stock_down

    # Calculate for bank
    _, emv_bank_up, _, utility_bank_up = calculate_second_period(stock_up=True)  # same as bank_down
    # EMV for choosing bank in the first period
    emv_bank_first = outcomes['bank_bank']
    # Utility for choosing bank in the first period
    utility_bank_first = utility(emv_bank_first)

    return emv_stock_first, emv_bank_first, utility_stock_first, utility_bank_first

# Calculate EMV and utility for the entire decision tree
def calculate_overall():
    emv_stock, emv_bank, utility_stock, utility_bank = calculate_first_period()
    
    # Choose the option with the higher EMV
    optimal_emv_action = 'stock' if emv_stock > emv_bank else 'bank'
    optimal_emv = max(emv_stock, emv_bank)

    # Choose the option with the higher utility
    optimal_utility_action = 'stock' if utility_stock > utility_bank else 'bank'
    optimal_utility = max(utility_stock, utility_bank)

    return {
        'optimal_emv_action': optimal_emv_action,
        'optimal_emv': optimal_emv,
        'optimal_utility_action': optimal_utility_action,
        'optimal_utility': optimal_utility,
        'emv_stock': emv_stock,
        'emv_bank': emv_bank,
        'utility_stock': utility_stock,
        'utility_bank': utility_bank
    }

# Execute the calculation
results = calculate_overall()
results


{'optimal_emv_action': 'stock',
 'optimal_emv': 11.280000000000001,
 'optimal_utility_action': 'stock',
 'optimal_utility': 0.10481668887539136,
 'emv_stock': 11.280000000000001,
 'emv_bank': 10.5,
 'utility_stock': 0.10481668887539136,
 'utility_bank': 0.10246950765959599}